In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv')
train1 = pd.read_csv('https://raw.githubusercontent.com/hritikzutshi/SocialMediaSentimentAnalysis/master/djangotest/proj/Corona_NLP_train.csv',encoding='latin1')

In [ ]:
train_original=train.copy()
train1_original=train1.copy()

In [ ]:
train.shape
train1.shape

In [ ]:
train_original


In [ ]:
train1_original

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/test.csv')
test1 = pd.read_csv('https://raw.githubusercontent.com/hritikzutshi/SocialMediaSentimentAnalysis/master/djangotest/proj/Corona_NLP_test.csv',encoding='latin1')

In [ ]:
test1

In [ ]:
train1.drop('TweetAt', inplace=True, axis=1)
train1.drop('ScreenName', inplace=True, axis=1)
train1.drop('Location', inplace=True, axis=1)

test1.drop('TweetAt', inplace=True, axis=1)
test1.drop('ScreenName', inplace=True, axis=1)
test1.drop('Location', inplace=True, axis=1)


In [ ]:
test1.head(10)

In [ ]:
train1['tweet'] = train1.OriginalTweet
train1["tweet"] = train1["tweet"].astype(str)
train1.drop('OriginalTweet', inplace=True, axis=1)


test1['tweet'] = test1.OriginalTweet
test1["tweet"] = test1["tweet"].astype(str)
test1.drop('OriginalTweet', inplace=True, axis=1)


def classes_def(x):
    if x ==  "Extremely Positive":
        return 1
    elif x == "Extremely Negative":
        return 0
    elif x == "Negative":
        return 0
    elif x ==  "Positive":
        return 1
    else:
        return 2
    

train1['label']=train1['Sentiment'].apply(lambda x:classes_def(x))
test1['label']=test1['Sentiment'].apply(lambda x:classes_def(x))

train1.label.value_counts(normalize= True)

train1.drop('Sentiment', inplace=True, axis=1)
test1.drop('Sentiment', inplace=True, axis=1)

train1 = train1.rename(columns={'UserName':'id'})
test1 = test1.rename(columns={'UserName':'id'})



In [ ]:
test1.head(10)

In [ ]:
train1.head(10)

In [ ]:
train1=train1[['id','label','tweet']]
test1=test1[['id','label','tweet']]

In [ ]:
test1.head(10)

In [ ]:
test_original=test.copy()
test1_original=test1.copy()

#### Here we see that there are a total of 17197 tweets in the test dataset

In [ ]:
test1.shape

In [ ]:
test1_original

### We combine Train and Test datasets for pre-processing stage

In [ ]:
# combine = train.append(test,ignore_index=True,sort=True)
# combine = combine.append(train1,ignore_index=True,sort=True)
# combine = combine.append(test1,ignore_index=True,sort=True)

combine = train.append(test,ignore_index=True,sort=True)
combine.isnull().sum().sort_values(ascending=False)



In [ ]:
combine.head()

In [ ]:
combine.tail()

In [ ]:
combine.shape

Given below is a user-defined function to remove unwanted text patterns from the tweets. It takes two arguments, one is the original string of text and the other is the pattern of text that we want to remove from the string. The function returns the same input string but without the given pattern. We will use this function to remove the pattern ‘@user’ from all the tweets in our data.


In [ ]:
def remove_pattern(text,pattern):    
    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)    
    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)    
    return text
    
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)    
        

In [ ]:
combine['tweet'] = np.vectorize(remove_pattern)(combine['tweet'], "@[\w]*")

combine['tweet']=combine['tweet'].apply(lambda x:remove_html(x))

combine['tweet']=combine['tweet'].apply(lambda x:remove_urls(x))

Punctuations, numbers and special characters do not help much. It is better to remove them from the text just as we removed the twitter handles. Here we will replace everything except characters and hashtags with spaces.

In [ ]:
combine['tweet'] = combine['tweet'].str.replace("[^a-zA-Z#]", " ", regex=True)

We have to be a little careful here in selecting the length of the words which we want to remove. So, I have decided to remove all the words having length 3 or less. For example, terms like “hmm”, “oh” are of very little use. It is better to get rid of them.

In [ ]:
combine['tweet'] = combine['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

Now we will tokenize all the cleaned tweets in our dataset. Tokens are individual terms or words, and tokenization is the process of splitting a string of text into tokens.

In [ ]:
tokenized_tweet = combine['tweet'].apply(lambda x: x.split())

Stemming is a rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc) from a word. For example, For example – “play”, “player”, “played”, “plays” and “playing” are the different variations of the word – “play”

In [ ]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])

tokenized_tweet.head()

#### Now let’s stitch these tokens back together.

In [ ]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combine['tweet'] = tokenized_tweet
combine.head()

#### Importing Packages necessary for generating a WordCloud

In [ ]:
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests

 #### Store all the words from the dataset which are non-racist/sexist

In [ ]:
all_words_positive = ' '.join(text for text in combine['tweet'][combine['label']==1])

#### We can see most of the words are positive or neutral. With happy, smile, and love being the most frequent ones. Hence, most of the frequent words are compatible with the sentiment which is non racist/sexists tweets. 

In [ ]:
# combining the image with the dataset
Mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png', stream=True).raw))

# We use the ImageColorGenerator library from Wordcloud 
# Here we take the color of the image and impose it over our wordcloud
image_colors = ImageColorGenerator(Mask)

# Now we use the WordCloud function from the wordcloud library 
wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask).generate(all_words_positive)

# Size of the image generated 
plt.figure(figsize=(10,20))

# Here we recolor the words from the dataset to the image's color
# recolor just recolors the default colors to the image's blue color
# interpolation is used to smooth the image generated 
plt.imshow(wc.recolor(color_func=image_colors),interpolation="hamming")

plt.axis('off')
plt.show()

#### Store all the words from the dataset which are racist/sexist

In [ ]:
all_words_negative = ' '.join(text for text in combine['tweet'][combine['label']==0])

#### As we can clearly see, most of the words have negative connotations. So, it seems we have a pretty good text data to work on.

In [ ]:
# combining the image with the dataset
Mask = np.array(Image.open(requests.get('http://clipart-library.com/image_gallery2/Twitter-PNG-Image.png', stream=True).raw))

# We use the ImageColorGenerator library from Wordcloud 
# Here we take the color of the image and impose it over our wordcloud
image_colors = ImageColorGenerator(Mask)

# Now we use the WordCloud function from the wordcloud library 
wc = WordCloud(background_color='black', height=1500, width=4000,mask=Mask).generate(all_words_negative)

# Size of the image generated 
plt.figure(figsize=(10,20))

# Here we recolor the words from the dataset to the image's color
# recolor just recolors the default colors to the image's blue color
# interpolation is used to smooth the image generated 
plt.imshow(wc.recolor(color_func=image_colors),interpolation="gaussian")

plt.axis('off')
plt.show()

### Function to extract hashtags from tweets

In [ ]:
def Hashtags_Extract(x):
    hashtags=[]
    
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r'#(\w+)',i)
        hashtags.append(ht)
    
    return hashtags

#### A nested list of all the hashtags from the positive reviews from the dataset

In [ ]:
ht_positive = Hashtags_Extract(combine['tweet'][combine['label']==0])

#### Here we unnest the list 

In [ ]:
ht_positive_unnest = sum(ht_positive,[])

#### A nested list of all the hashtags from the negative reviews from the dataset

In [ ]:
ht_negative = Hashtags_Extract(combine['tweet'][combine['label']==1])

#### Here we unnest the list

In [ ]:
ht_negative_unnest = sum(ht_negative,[])

#### Counting the frequency of the words having Positive Sentiment 

In [ ]:
word_freq_positive = nltk.FreqDist(ht_positive_unnest)

word_freq_positive

#### Creating a dataframe for the most frequently used words in hashtags

In [ ]:
df_positive = pd.DataFrame({'Hashtags':list(word_freq_positive.keys()),'Count':list(word_freq_positive.values())})

In [ ]:
df_positive.head(10)

#### Plotting the barplot for the 10 most frequent words used for hashtags 

In [ ]:
df_positive_plot = df_positive.nlargest(20,columns='Count') 

In [ ]:
sns.barplot(data=df_positive_plot,y='Hashtags',x='Count')
sns.despine()

#### Counting the frequency of the words having Negative Sentiment 

In [ ]:
word_freq_negative = nltk.FreqDist(ht_negative_unnest)

In [ ]:
word_freq_negative

#### Creating a dataframe for the most frequently used words in hashtags

In [ ]:
df_negative = pd.DataFrame({'Hashtags':list(word_freq_negative.keys()),'Count':list(word_freq_negative.values())})

In [ ]:
df_negative.head(10)

#### Plotting the barplot for the 10 most frequent words used for hashtags 

In [ ]:
df_negative_plot = df_negative.nlargest(20,columns='Count') 

In [ ]:
sns.barplot(data=df_negative_plot,y='Hashtags',x='Count')
sns.despine()

### Bag-of-Words Features

### TF-IDF Features

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

#combine = clean_dataset(combine)
np.nan_to_num(combine)

# Applying Machine Learning Models

### Using features from TF-IDF for training set

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(sublinear_tf=True, min_df=5,stop_words='english')

tfidf_matrix=tfidf.fit_transform(combine['tweet'])

df_tfidf = pd.DataFrame(tfidf_matrix.todense())

df_tfidf

In [ ]:
tfidf_matrix = tfidf_matrix[:31962]
np.any(np.isnan(df_tfidf))

### Splitting the data into training and validation set

In [ ]:
from sklearn.model_selection import train_test_split

#### Using TF-IDF features

In [ ]:
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(tfidf_matrix,train['label'],test_size=0.3,random_state=42)


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
Log_Reg = LogisticRegression(solver = 'liblinear', random_state = 42, max_iter=1000)

#### Calculating the F1 score

### Using TF-IDF Features

In [ ]:
Log_Reg.fit(x_train_tfidf,y_train_tfidf)

In [ ]:
prediction_tfidf = Log_Reg.predict(x_valid_tfidf)

prediction_tfidf

#### Calculating the F1 score and Accuracy

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [ ]:
# prediction_tfidf=prediction_tfidf[:,1]>=0.3

# # converting the results to integer type
# prediction_int_tfidf=prediction_tfidf.astype(np.int)


In [ ]:
# calculating f1 score
log_score = f1_score(y_valid_tfidf, prediction_tfidf)
log_score

In [ ]:
log_accuracy = accuracy_score(y_valid_tfidf,prediction_tfidf)
log_accuracy

## XGBoost

In [ ]:
from xgboost import XGBClassifier

### Using TF-IDF Features 

In [ ]:
model_tfidf=XGBClassifier(random_state=29,learning_rate=0.7)

In [ ]:
model_tfidf.fit(x_train_tfidf, y_train_tfidf)

In [ ]:
# The first part of the list is predicting probabilities for label:0 
# and the second part of the list is predicting probabilities for label:1
xgb_tfidf=model_tfidf.predict(x_valid_tfidf)

xgb_tfidf

#### Calculating the F1 score and accuracy


In [ ]:
# if prediction is greater than or equal to 0.3 than 1 else 0
# Where 0 is for positive sentiment tweets and 1 for negative sentiment tweets
# xgb_tfidf=xgb_tfidf[:,1]>=0.3

# # converting the results to integer type
# xgb_int_tfidf=xgb_tfidf.astype(np.int)

# calculating f1 score
xbg_score=f1_score(y_valid_tfidf,xgb_tfidf)

xbg_score

In [ ]:
xbg_accuracy = accuracy_score(y_valid_tfidf,xgb_tfidf)
xbg_accuracy

## Random Forest

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)

### Using TF-IDF Features

In [ ]:
rfc.fit(x_train_tfidf,y_train_tfidf)

In [ ]:
rfc_pred = rfc.predict(x_valid_tfidf)

rfc_pred

#### Calculating F1 Score and accuracy

In [ ]:
# if prediction is greater than or equal to 0.3 than 1 else 0
# Where 0 is for positive sentiment tweets and 1 for negative sentiment tweets
# rfc_pred=rfc_pred[:,1]>=0.3

# # converting the results to integer type
# rfc_int_tfidf=rfc_pred.astype(np.int)

# calculating f1 score
rfc_score=f1_score(y_valid_tfidf,rfc_pred)
rfc_score

In [ ]:
rfc_accuracy = accuracy_score(y_valid_tfidf, rfc_pred )
rfc_accuracy

# Multinomial Naive Bayes


In [ ]:
nb = MultinomialNB()
nb.fit(x_train_tfidf,y_train_tfidf)

In [ ]:
nb_model = nb.predict(x_valid_tfidf)
nb_model

### Calculating F1 Score and accuracy

In [ ]:
# nb_model=nb_model[:,1]>=0.3

# # converting the results to integer type
# nb_model=nb_model.astype(np.int)

# calculating f1 score
naive_bays_score=f1_score(y_valid_tfidf,nb_model)

naive_bays_score

In [ ]:
naive_bays_accuracy = accuracy_score(y_valid_tfidf,nb_model)
naive_bays_accuracy

# Linear support vector

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
lsvc = LinearSVC()
lsvc.fit(x_train_tfidf,y_train_tfidf)

In [ ]:
# lsvc_new = CalibratedClassifierCV(lsvc,cv='prefit')
# lscv_new = lsvc_new.fit(x_valid_tfidf,y_valid_tfidf) 
lsvc_model = lsvc.predict(x_valid_tfidf)
lsvc_model

### Calculating F1 Score and accuracy

In [ ]:
# lsvc_model=lsvc_model[:,1]>=0.3

# # converting the results to integer type
# lsvc_model=lsvc_model.astype(np.int)

# calculating f1 score
lsvc_score=f1_score(y_valid_tfidf,lsvc_model)

lsvc_score

In [ ]:
lsvc_accuracy = accuracy_score(y_valid_tfidf,lsvc_model)
lsvc_accuracy

# Model Comparison

In [ ]:
Algo=['LogisticRegression','XGBoost','RandomForest','NaiveBays','LinearSVC']

In [ ]:
score = [log_score,xbg_score,rfc_score,naive_bays_score,lsvc_score]

compare=pd.DataFrame({'Model':Algo,'F1-Score':score},index=[i for i in range(1,6)])

In [ ]:
compare.T

In [ ]:
plt.figure(figsize=(18,5))

sns.pointplot(x='Model',y='F1-Score',data=compare)

plt.title('Model Vs F1-Score')
plt.xlabel('MODEL')
plt.ylabel('F1-Score')

plt.show()

In [ ]:
accuracy = [log_accuracy,xbg_accuracy,rfc_accuracy,naive_bays_accuracy,lsvc_accuracy]

compare=pd.DataFrame({'Model':Algo,'Accuracy':accuracy},index=[i for i in range(1,6)])

In [ ]:
plt.figure(figsize=(18,5))

sns.pointplot(x='Model',y='Accuracy',data=compare)

plt.title('Model Vs Accuracy')
plt.xlabel('MODEL')
plt.ylabel('Accuracy')

plt.show()

In [ ]:
compare.T

## Using the best possible model to predict for the test data

#### From the above comaprison graph we can see that Logistic Regression trained using TF-IDF features gives us the best performance

In [ ]:
test_tfidf = tfidf_matrix[31962:]

In [ ]:
test_pred = Log_Reg.predict_proba(test_tfidf)

test_pred_int = test_pred[:,1] >= 0.3

test_pred_int = test_pred_int.astype(np.int)

test['label'] = test_pred_int

submission = test[['id','label']]

submission.to_csv('result.csv', index=False)

### Test dataset after prediction

In [ ]:
res = pd.read_csv('result.csv')

In [ ]:
res

In [ ]:
sns.countplot(train_original['label'])
sns.despine()